In [45]:
import pandas as pd
import numpy as np
import random

from model import Patient

random_seed = 42

In [6]:
def patient_to_row(patient):
    return [patient.patient_id, """CKD""", patient.sex, patient.age, ]
    

In [42]:
train_ids = train_ids['Patient'].to_list()

In [46]:
random.seed(random_seed)
random.shuffle(train_ids)

In [41]:
train_ids_pos = pd.read_csv('../../data/TRN/pos_trn.csv')
train_ids_neg = pd.read_csv('../../data/TRN/neg_trn.csv')
train_ids_neg.rename(columns={'Patient_id': 'Patient'}, inplace=True)
train_ids = pd.concat([train_ids_pos, train_ids_neg])
train_ids['Patient'] = train_ids['Patient'].astype(int)

In [48]:
def patients_to_df(id_list):
    columns=['ID', 'CDK', 'sex', 'age', 'last_eGFR', 'avg_eGFR', 'last_uACR', 'avg_uACR']